create a comment toxicity model 

insatll dependencies 

In [ ]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn


  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "C:\Python38\lib\site-packages\packaging\requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
        File "C:\Python38\lib\site-packages\packaging\_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "C:\Python38\lib\site-packages\packaging\_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "C:\Python38\lib\site-packages\packaging\_parser.py", line 126, in _parse_requirement_details
          marker = _parse_requirement_marker(
        File "C:\Python38\lib\site-packages\packaging\_parser.py", line 147, in _parse_requirement_marker
          tokenizer.raise_syntax_error(
        File "C:\Python38

In [2]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
df = pd.read_csv(os.path.join('C:\\Users\\elban\\Downloads\\CommentToxicity-main\\CommentToxicity-main\\jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))


In [4]:
df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


1. Preprocess 

In [5]:
!pip list

Package                                  Version
---------------------------------------- --------------------
absl-py                                  2.1.0
accelerate                               0.34.2
aiohappyeyeballs                         2.4.2
aiohttp                                  3.10.8
aiosignal                                1.3.1
altair                                   4.2.0
annotated-types                          0.7.0
anyio                                    4.5.0
asgiref                                  3.8.1
astunparse                               1.6.3
async-timeout                            4.0.3
attrs                                    24.2.0
backoff                                  2.2.1
backports.zoneinfo                       0.2.1
bcrypt                                   4.2.0
beautifulsoup4                           4.12.3
bitsandbytes                             0.44.0
blinker                                  1.8.2
build                                 

###  vectorization data

In [6]:
from tensorflow.keras.layers import TextVectorization


In [7]:

X = df['comment_text']
y = df[df.columns[2:]].values

In [8]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [9]:
MAX_FEATURES = 200000 # number of words in the vocab


### tokenization

In [10]:

vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [11]:
vectorizer.adapt(X.values)


In [12]:
vectorized_text = vectorizer(X.values)


In [13]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

### crating a dataset 

In [14]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [15]:
dataset.as_numpy_iterator().next()

(array([[     8,   1561,      5, ...,      0,      0,      0],
        [    29,   7087,      9, ...,      0,      0,      0],
        [   433,   6949,    218, ...,      0,      0,      0],
        ...,
        [    12,      9,     15, ...,      0,      0,      0],
        [   104, 128420,      5, ...,      0,      0,      0],
        [   191,    288,      5, ...,      0,      0,      0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

### split model (train, val,test)

In [16]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

2. create sequential model

In [17]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

* The embedding layer helps the model learn relationships between words or tokens by mapping each one to a vector in a lower-dimensional space.which converts input integer tokens into dense vectors of fixed size.

* This model is designed for a multi-label text classification task. The embedding layer helps the model understand word semantics, the LSTM captures sequential information, and the dense layers learn additional patterns. The final layer with sigmoid activation outputs predictions for multiple classes, which is common in cases where a single input may belong to multiple categories simultaneously.


MAX_FEATURES+1 : for extra token : padding


In [18]:

model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))

# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))

# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [19]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')


In [22]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1800, 32)       │     6,400,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,475,060 (74.29 MB)

 Trainable params: 6,491,686 (24.76 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,983,374 (49.53 MB)

In [ ]:
history = model.fit(train, epochs=5, validation_data=val)


Epoch 1/5
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step - loss: 0.0718

In [ ]:
from matplotlib import pyplot as plt


In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

NameError: name 'history' is not defined

<Figure size 800x500 with 0 Axes>

3.make predictions 

In [ ]:
input_text = vectorizer('You freaking suck!')


In [ ]:
res = model.predict(input_text)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_1_1/Cast:0", shape=(32,), dtype=float32). Expected shape (None, 1800), but input has incompatible shape (32,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32,), dtype=int64)
  • training=False
  • mask=None

In [ ]:
(res > 0.5).astype(int)


In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()


In [ ]:
(model.predict(batch_X) > 0.5).astype(int)


In [ ]:
res.shape


4. evaluate model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy


In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 

    # Unpack the batch 
    X_true, y_true = batch
    
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')


5. test

In [ ]:
!pip install gradio jinja2


In [ ]:
import gradio as gr

In [ ]:
model.save('toxicity.h5')


In [ ]:
model = tf.keras.models.load_model('toxicity.h5')


In [ ]:
input_str = vectorizer('hey i freaken hate you!')


In [ ]:
res = model.predict(np.expand_dims(input_str,0))


In [ ]:
res


In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)